In [1]:
# Install required libraries
!pip install --upgrade pip
!pip install "unsloth[cu118-torch261] @ git+https://github.com/unslothai/unsloth.git"
!pip install transformers datasets bitsandbytes accelerate peft evaluate wandb pandas streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-7zqt17l9/unsloth_91f0a42c5c6b4e8b8e4d4c6b6683a4b6
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-7zqt17l9/unsloth_91f0a42c5c6b4e8b8e4d4c6b6683a4b6
  Resolved https://github.com/unslothai/unsloth.git to commit bcb1bcfd6aa9dc7db8910c733cecf0c676b37eb3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.8.9-py3-none-any.whl size=312437 sha256=fa254c84007c8bd8a4093ad3718e306b77a30e65e57b50f63fe66d042229398a
  Stored in directory: /tmp/pip-ephem-wheel-cache-n7got2ym/wheels/60/3e/1f/e576c07051d90cf64b6a41434d8

In [3]:
pip install unsloth_zoo

  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [unsloth_zoo]


In [4]:
import torch
from unsloth import FastLanguageModel

model_path = "/content/drive/MyDrive/tinyllama-it-support-lora"

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=512,
    dtype=torch.float16,
    load_in_4bit=True,        # LoRA + 4-bit quantized
    device_map="auto"         # automatically uses GPU if available
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Unsloth 2025.8.9 patched 22 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [5]:
class ITHelpBotAgent:
    def __init__(self, model, tokenizer, max_context=5):
        self.model = model
        self.tokenizer = tokenizer
        self.context = []
        self.max_context = max_context

    def ask(self, user_input):
        # Append user query
        self.context.append({"role": "user", "text": user_input})

        # Build prompt with last N turns
        prompt = "### Conversation:\n"
        for turn in self.context[-self.max_context:]:
            prompt += f"{turn['role'].capitalize()}: {turn['text']}\n"
        prompt += "### Response:\n"

        # Tokenize and generate
        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(self.model.device)
        outputs = self.model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=self.tokenizer.eos_token_id
        )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        if "### Response:" in response:
            response = response.split("### Response:")[-1].strip()

        # Append agent response to context
        self.context.append({"role": "agent", "text": response})
        return response

# Initialize agent
agent = ITHelpBotAgent(model=model, tokenizer=tokenizer)


In [8]:
# Initialize agent
agent = ITHelpBotAgent(model=model, tokenizer=tokenizer)

# Example queries
queries = [
    "My Zoom audio is not working, what should I do?",
    "Outlook is not sending emails, how can I fix it?",
    "I cannot connect to the VPN, it keeps failing."
]

# Get responses
for q in queries:
    answer = agent.ask(q)
    print(f"Q: {q}\nA: {answer}\n{'-'*50}")


Q: My Zoom audio is not working, what should I do?
A: Check your internet connection, restart the VPN client, and ensure your credentials are correct. You can reset your Windows login password by pressing Ctrl+Alt+Del, selecting 'Change a password', and following the prompts. Check Task Manager for high usage apps, clear temporary files, and restart your laptop.
--------------------------------------------------
Q: Outlook is not sending emails, how can I fix it?
A: Restart your laptop, restart Zoom, and check microphone settings. Restart Zoom, select the Zoom client, and click on 'Zoom' in the top-right corner of the screen. Select 'Update' from the drop-down menu. Open the antivirus software and scan for malware. If you still experience issues, contact Zoom customer support for assistance.
--------------------------------------------------
Q: I cannot connect to the VPN, it keeps failing.
A: Check your internet connection, restart Zoom, and open the VPN configuration page. Select the